This notebook allows to compute different feature from the dataset.

IMPORTANT you have to launch the sementation file in order to compute the test seg with the left ventricule label. This new seg is then stored in Dataset/SegTest.

 b. **Extraction des features (py-radiomics)**  
   - Activez ou désactivez les classes de features souhaitées en commentant/décommentant les lignes ci-dessous :

     ```python
     # Features à extraire :
     extractor.enableFeatureClassByName('shape')      # Forme (volume, sphéricité…)
     extractor.enableFeatureClassByName('firstorder') # Statistiques de premier ordre
     extractor.enableFeatureClassByName('glcm')       # Gray Level Co-occurrence Matrix
     extractor.enableFeatureClassByName('glrlm')      # Gray Level Run Length Matrix
     ```

   - Les jeux de données extraits seront sauvegardés dans :
     - `data/shape_glcm_features` si vous avez sélectionné `shape` et `glcm`
     - `data/shape_firstorder_glrlm_features` si vous avez désélectionné `glrlm`

voir le rapport sur le detail des features extraites ou documentation py-radiomics

In [1]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk
from radiomics import featureextractor
import itertools
import re


In [2]:
# Py-radiomics extractor that extract radiomic features.
extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.settings['normalize'] = True             # Z-score normalization
extractor.settings['normalizeScale'] = 100         # Stretch intensity to approx -300..+300
extractor.settings['removeOutliers'] = 3           # Clip extreme values beyond ±3σ
extractor.settings['voxelArrayShift'] = 300        # Shift to make intensities all positive

# === Discretization for texture ===
extractor.settings['binWidth'] = 5                 # Recommended for MRI after scaling

# === Resampling settings ===
extractor.settings['resampledPixelSpacing'] = [2, 2, 2]  # Isotropic 2mm resolution
extractor.settings['interpolator'] = 'sitkBSpline'       # Smooth interpolation for MRI

extractor.settings['additionalInfo'] = False
extractor.disableAllImageTypes()
extractor.enableImageTypeByName('Original')
extractor.disableAllFeatures()

# Feature que l'on veut : 
extractor.enableFeatureClassByName('shape') # Some shape features (volume, sphericity etc)
#extractor.enableFeatureClassByName('firstorder') # Some signal features
extractor.enableFeatureClassByName("glcm")
extractor.enableFeatureClassByName("glrlm")


In [3]:
# I just did that to be have clean folder depending on feature selected.
output_string = ""
for cls, feats in extractor.enabledFeatures.items():
    output_string += f"{cls}_" 
output_string += "features"
    

In [4]:
print(output_string)

shape_glcm_glrlm_features


In [5]:
BASE       = os.getcwd()
TRAIN_DIR  = os.path.join(BASE, "Dataset", "Train")
TEST_DIR   = os.path.join(BASE, "Dataset", "SegTest")
OUT_DIR    = os.path.join(BASE, "data",output_string)
os.makedirs(OUT_DIR, exist_ok=True)

In [6]:
def extract_per_patient(folder, sid):
    rec = {"Id": sid}
    for ph in ["ED", "ES"]:
        img = sitk.ReadImage(os.path.join(folder, f"{sid}_{ph}.nii"))
        msk = sitk.ReadImage(os.path.join(folder, f"{sid}_{ph}_seg.nii"))
        for lbl,name in [(1,"RV"),(3,"LV"),(2,"MY")]:
            try:
                feats = extractor.execute(img, msk, label=lbl)
                for k,v in feats.items():
                    rec[f"{ph}_{name}_{k.replace('original_','')}"] = v
            except ValueError:
                # si jamais pas de label
                # avec nos donnés en pratique on passe jamais dedans.
                for k in extractor.enabledFeatures['shape']:
                    rec[f"{ph}_{name}_{k}"] = np.nan
    return rec

def all_patient(src_dir):
    # First i forgot to sort that lead to indexes problem
    patient_folders = sorted(
        [s for s in os.listdir(src_dir) if os.path.isdir(os.path.join(src_dir, s))],
        key=lambda x: int(x)
    )

    # Extract features for each patient in the right order
    rows = [extract_per_patient(os.path.join(src_dir, s), s) for s in patient_folders]

    df = pd.DataFrame(rows)
    return df

def add_ratios(df,Train=True):
    
    # on récupère uniquement les features qui correspondent à des volume.
    # Dans les différents papiers la fractions d' ejection qui est un rapport revient souvent comme une feature importante. 
    # D'où l'idée de calculer différente ratio
    
    vol_cols = [c for c in df.columns if re.search(r'VoxelVolume$', c)]
    print(vol_cols)
    
    # on utilise combinations et pas product de itertools pour ne pas calculer A/B et B/A.
    # Car ce n'est pas très utile d'avoir des features redondantes comme ça
    ratio_frames = {}         

    for a,b in itertools.product(vol_cols, repeat = 2):
        if a != b :
            name = f"{a}_over_{b}"              
            ratio = np.divide(df[a], df[b])
            ratio_frames[name] = ratio

    # merge 
    df = pd.concat([df, pd.DataFrame(ratio_frames)], axis=1)

    return df
 

def compute_body_surface_area(height,weight): 
    "Return the body surface area from height and weight. (Formula of Du Bois)"
    
    return 0.007184 * (height**0.725 )* (weight**0.425)

def add_body_surface_area_feature(df : pd.DataFrame ,name_column_height = "Height",name_column_weight = "Weight"):
    # Add for each row the BSA associated.
    
    if (name_column_height and name_column_weight in df.columns) and ("body_surface" not in df.columns)  :
        df["body_surface"] = compute_body_surface_area(df[name_column_height],df[name_column_weight])
        #print("Body surface area feature added")
    else : 
        print("provide a dataframe with a height and weight feature")
       

def add_meta_data(df,Train= True):  
    if Train : 
        metaData = pd.read_csv(os.path.join(BASE,"Dataset","metaDataTrain.csv"))
        metaData.drop(columns=["Category"],inplace=True)
        # print(metaData.head())
    else :
        metaData = pd.read_csv(os.path.join(BASE,"Dataset","metaDataTest.csv"))
        # print(metaData.head())
    
    # I had some issues with int index as 001 can be translated to 1..
    # So I needed to ensure all Id have the same size by filling from the left 0s .
    df["Id"] = df["Id"].astype(str).str.zfill(3)  
    metaData["Id"] = metaData["Id"].astype(str).str.zfill(3)
    df = df.merge(metaData, on="Id", how="left")
    
    add_body_surface_area_feature(df)           
    df.drop(columns=["Height", "Weight"], errors="ignore", inplace=True)
    
    return df

In [7]:

df_train_all_features = all_patient(TRAIN_DIR)
df_test_all_features = all_patient(TEST_DIR)


NiftiImageIO (0x10b14e0c0): /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/Dataset/Train/001/001_ED.nii has unexpected scales in sform

NiftiImageIO (0x10b14e0c0): /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/Dataset/Train/001/001_ED.nii has unexpected scales in sform

NiftiImageIO (0x10fea92d0): /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/Dataset/Train/001/001_ED_seg.nii has unexpected scales in sform

NiftiImageIO (0x10fea92d0): /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/Dataset/Train/001/001_ED_seg.nii has unexpected scales in sform

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
NiftiImageIO (0x10a8c9690)

In [8]:

df_train = add_ratios(df_train_all_features,Train=True)
df_test = add_ratios(df_test_all_features,Train=False)


['ED_RV_shape_VoxelVolume', 'ED_LV_shape_VoxelVolume', 'ED_MY_shape_VoxelVolume', 'ES_RV_shape_VoxelVolume', 'ES_LV_shape_VoxelVolume', 'ES_MY_shape_VoxelVolume']
['ED_RV_shape_VoxelVolume', 'ED_LV_shape_VoxelVolume', 'ED_MY_shape_VoxelVolume', 'ES_RV_shape_VoxelVolume', 'ES_LV_shape_VoxelVolume', 'ES_MY_shape_VoxelVolume']


In [9]:

df_train = add_meta_data(df_train,Train=True)
df_test = add_meta_data(df_test,Train=False)


In [10]:
# Création du label set
y = pd.read_csv(os.path.join(BASE,"Dataset","metaDataTrain.csv"), usecols=["Id", "Category"])
y["Id"] = y["Id"].astype(str).str.zfill(3)  # Pour correspondre à df_train

# Alignement de y 
y_aligned = y.set_index("Id").loc[df_train["Id"]].reset_index()

df_train.to_csv(os.path.join(OUT_DIR,"TrainningDataset.csv"), index=False)
df_test.to_csv(os.path.join(OUT_DIR,"TestingDataset.csv"), index=False)
y_aligned.to_csv(os.path.join(OUT_DIR,"TrainningDatasetCategory.csv"), index=False)
